In [121]:
import torch
import numpy as np
from torch.utils.tensorboard import SummaryWriter

In [122]:
## A pytorch tensor (random)
a = torch.FloatTensor(3, 2)
a


tensor([[2.9147e-43, 0.0000e+00],
        [6.5160e-43, 0.0000e+00],
        [1.0507e+37, 0.0000e+00]])

In [123]:
## clear content of a tensor (without altering the tensor) (Functional, makes a copy
## of the tensor)
b = torch.zeros_like(a)
print(a)
print(b)
## clear content of a tensor (alter the tensor)
a.zero_()
print(a)

tensor([[2.9147e-43, 0.0000e+00],
        [6.5160e-43, 0.0000e+00],
        [1.0507e+37, 0.0000e+00]])
tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])
tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])


In [124]:
a_numpy = np.random.uniform((3,2))
a_tensor = torch.tensor(a_numpy, dtype=torch.float32)
a_tensor

tensor([1.4638, 1.6417])

In [125]:
## tensor operations
a = torch.tensor([1., 2., 3.])
print(a.sum())
print(a.mean())
print(a.mode())
print(a.median())



tensor(6.)
tensor(2.)
torch.return_types.mode(
values=tensor(1.),
indices=tensor(0))
tensor(2.)


In [126]:
## GPU tensors
print(a)
print(a.device)
print(a.to('cuda'))
print(a.to('cuda').device)

tensor([1., 2., 3.])
cpu
tensor([1., 2., 3.], device='cuda:0')
cuda:0


In [127]:
## tensor gradients
v1 = torch.tensor([1.0, 2.0])
v2 = torch.tensor([3.1, 1.0], requires_grad=True)

## operate on the tensors
v_sum = v1 + v2
v_res = (v_sum*2).sum()
print(v_sum)
print(v_res)

## user made tensors are leafs in  a graph
## but intermediate operations are not
print(v1.is_leaf)
print(v_sum.is_leaf)

## are gradients required
print(v1.requires_grad)
print(v2.grad)

tensor([4.1000, 3.0000], grad_fn=<AddBackward0>)
tensor(14.2000, grad_fn=<SumBackward0>)
True
False
False
None


In [128]:
## NN blocks
import torch.nn as nn
linear_layer = nn.Linear(3, 5, bias=False)
inp = torch.tensor([1, 2, 3], dtype=torch.float)
linear_layer(inp)

tensor([-1.4658,  1.8997, -1.2867,  0.4601,  0.4875],
       grad_fn=<SqueezeBackward3>)

In [129]:
## layer parameter
for i in linear_layer.parameters():
    print(i)

Parameter containing:
tensor([[ 0.0476, -0.2724, -0.3229],
        [ 0.2363,  0.5315,  0.2001],
        [ 0.2612, -0.1356, -0.4256],
        [-0.4980, -0.1145,  0.3957],
        [ 0.1956,  0.5295, -0.2557]], requires_grad=True)


In [130]:
## layer parameter
print(linear_layer.state_dict())

OrderedDict([('weight', tensor([[ 0.0476, -0.2724, -0.3229],
        [ 0.2363,  0.5315,  0.2001],
        [ 0.2612, -0.1356, -0.4256],
        [-0.4980, -0.1145,  0.3957],
        [ 0.1956,  0.5295, -0.2557]]))])


In [131]:
## A simple ANN model
model = nn.Sequential(
    nn.Linear(2,5),
    nn.ReLU(),
    nn.Linear(5,20),
    nn.ReLU(),
    nn.Linear(20,10),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Softmax(dim=1)
)
model

Sequential(
  (0): Linear(in_features=2, out_features=5, bias=True)
  (1): ReLU()
  (2): Linear(in_features=5, out_features=20, bias=True)
  (3): ReLU()
  (4): Linear(in_features=20, out_features=10, bias=True)
  (5): ReLU()
  (6): Dropout(p=0.3, inplace=False)
  (7): Softmax(dim=1)
)

In [132]:
model(torch.FloatTensor([[1,2]]))

tensor([[0.0916, 0.0916, 0.1130, 0.0916, 0.1110, 0.0916, 0.0916, 0.0916, 0.1145,
         0.1119]], grad_fn=<SoftmaxBackward>)

In [133]:
## model construction by subclassing
class Model(nn.Module):
    def __init__(self, input_shape, output_shape, drop=0.3):
        super(Model, self).__init__()
        self.model = nn.Sequential(
                        nn.Linear(2,5),
                        nn.ReLU(),
                        nn.Linear(5,20),
                        nn.ReLU(),
                        nn.Linear(20,1),
                        #nn.ReLU(),
                        #nn.Dropout(p=0.3),
                        #nn.Softmax(dim=1)
                        )
    ## forward pass of the NN
    def forward(self, x):
        return self.model(x)


In [134]:
## construct the model
net = Model(2, 3)
inp = torch.FloatTensor([[2, 3]])
out = net(inp)
print(net)
print(out)

Model(
  (model): Sequential(
    (0): Linear(in_features=2, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)
tensor([[0.0050]], grad_fn=<AddmmBackward>)


In [135]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.001)

In [136]:
## make some random data
inp_batch = torch.normal(0, 1, (1000, 2))
inp_batch.size()
tgt_batch = torch.normal(0, 1, (1000, 1))

In [137]:
writer = SummaryWriter()
for e in range(100):
    epoch_loss = list()
    for i in range(0, inp_batch.shape[0], 10):
        inp = inp_batch[i:i+10]
        tgt = tgt_batch[i:i+10]
        out = net(inp)
        #print(out.shape)
        optimizer.zero_grad()
        loss = nn.MSELoss()(out, tgt)
        loss.backward()
        optimizer.step()
        epoch_loss.append(loss.item())
        writer.add_scalar("error", loss.item())
    print(np.array(np.mean(epoch_loss)))
writer.close()

    

AttributeError: 'float' object has no attribute 'type'